In [4]:
from src.dpsk_modem import *
from src.ofdm_modem import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [5]:
# settings
np.random.seed(0)
fs_hz = 1500
scs_hz = 15
num_sc = 64
num_ofdm_symbols = 4
bits_per_sample = 1 # [dbpsk: 1, dqpsk: 2, d8psk: 3]
num_transmit_bits = (num_sc-1) * num_ofdm_symbols * bits_per_sample

# modem block init
dpsk_block = DPSKModem(bits_per_sample=bits_per_sample)
ofdm_block = OFDMModem(num_sc=num_sc)

# FFT settings (for spectra)
n_fft = 4 * (num_transmit_bits // dpsk_block.bits_per_sample * int(fs_hz / scs_hz))
freqs = np.linspace(-fs_hz/2, fs_hz/2, n_fft)

In [6]:
def est_papr_db(signal: np.ndarray):
    papr = np.max(np.abs(signal)**2) / np.mean(np.abs(signal)**2)
    return 10*np.log10(papr)

In [8]:
# processing
tx_bits = np.random.randint(low=0, high=2, size=num_transmit_bits)
tx_signal = ofdm_block.process_tx(tx_bits, dpsk_block, num_ofdm_symbols)
rx_bits = ofdm_block.process_rx(tx_signal, dpsk_block, num_ofdm_symbols)
print('Tx bits: ', *tx_bits[:min(10, len(tx_bits))])
print('Rx bits: ', *rx_bits[:min(10, len(tx_bits))])
print("Correct: ", np.array_equal(rx_bits, tx_bits))
print("Tx signal PAPR [dB]: ", np.round(est_papr_db(tx_signal), 1))

Tx bits:  1 0 1 1 1 1 0 0 1 1
Rx bits:  1 0 1 1 1 1 0 0 1 1
Correct:  True
Tx signal PAPR [dB]:  7.9
